In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import dotenv
from huggingface_hub import login
import torch
import transformers
import time

_ = dotenv.load_dotenv(dotenv_path=".env", override=True)
login(token=os.environ["HF_TOKEN"])

model_name = "google/gemma-3-1b-it"
local_path = "./models"
os.makedirs(local_path, exist_ok=True)

start_time = time.time()
model = transformers.AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    token=os.environ["HF_TOKEN"],
    device_map="auto", dtype=torch.bfloat16
)
model.save_pretrained(save_directory=local_path)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name,
    token=os.environ["HF_TOKEN"]
)
tokenizer.save_pretrained(save_directory=local_path)
end_time = time.time() - start_time

print("Time taken: %.2f"%(end_time))

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

local_dir = "./models"
llm = HuggingFacePipeline.from_model_id(task="text-generation", model_id=local_dir,
    pipeline_kwargs={"max_new_tokens": 512, "temperature": 0.1})
llm = ChatHuggingFace(llm=llm)

## Customer Review

In [2]:
from langchain_core import prompts
import time

review_template = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else?
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

prompt_template = prompts.PromptTemplate.from_template(template=review_template)

chain = prompt_template | llm.bind(skip_prompt=True)

customer_review = """
This leaf blower is pretty amazing.  It has four settings:
candle blower, gentle breeze, windy city, and tornado. It
arrived in two days, just in time for my wife's anniversary
present. I think my wife liked it so much she was speechless. So
far I've been the only one using it, and I've been using it
every other morning to clear the leaves on our lawn. It's
slightly more expensive than the other leaf blowers out there,
but I think it's worth it for the extra features.
"""

start_time = time.time()
review_response = chain.invoke(input={"text": customer_review})
end_time = time.time() - start_time
print("Duration: %.2f, Response: %s"%(end_time, review_response.content))

Duration: 31.50, Response: ```json
{
  "gift": true,
  "delivery_days": "2",
  "price_value": "leaf blower"
}
```


In [3]:
from typing import TypedDict
from langgraph import graph
import time

class GraphState(TypedDict):
    text: str
    response: dict

def review_node(state: GraphState):
    response = chain.invoke(input={"text": state["text"]})
    return {"response": response}

workflow = graph.StateGraph(state_schema=GraphState)
workflow.add_node(node="review", action=review_node)
workflow.add_edge(start_key=graph.START, end_key="review")
workflow.add_edge(start_key="review", end_key=graph.END)
agents = workflow.compile()

start_time = time.time()
review_response = agents.invoke(input={"text": customer_review})
end_time = time.time() - start_time
print("Duration: %.2f, Response: %s"%(end_time, review_response["response"].content))

Duration: 29.72, Response: ```json
{
  "gift": true,
  "delivery_days": "2",
  "price_value": "leaf blower"
}
```


## Langgraph

In [2]:
# Define tools
from langchain.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply `a` and `b`.
    Args:
        a: First int
        b: Second int
    """
    return a * b

@tool
def add(a: int, b: int) -> int:
    """
    Adds `a` and `b`.
    Args:
        a: First int
        b: Second int
    """
    return a + b

@tool
def divide(a: int, b: int) -> float:
    """
    Divide `a` and `b`.
    Args:
        a: First int
        b: Second int
    """
    return a / b

tools = [add, multiply, divide]
llm_with_tools = llm.bind_tools(tools=tools)

In [9]:
# Define state
import typing
from langchain import messages
import operator

class MessagesState(typing.TypedDict):
    messages: typing.Annotated[list[messages.AnyMessage], operator.add]
    llm_calls: int

In [11]:
# Define llm node
from langchain import messages

tools = [wikipedia_tool]
llm_with_tools = llm.bind_tools(tools=tools)

def llm_call(state: dict):
    """
    LLM decides whether to call a tool or not.
    """
    return {
        "messages": [
            llm_with_tools.invoke(input=[
                messages.SystemMessage(content="""
                    You're a helpful assistant tasked with performing
                    Wikipedia search on inputs.
                """)            
            ] + state["messages"])
        ],
        "llm_calls": state.get("llm_calls", 0) + 1
    }

In [6]:
%%bash
pip install --no-cache-dir -qU \
    wikipedia

In [ ]:
# Define wikipedia tools
from langchain import tools
import wikipedia

@tools.tool
def wikipedia_tool(
    query: str
):
    """
    Use `query` to search Wikipedia for factual information.
    Args:
        query: str
    """
    try:
        title = wikipedia.search(query)[0]
        if not title:
            return "No results found on Wikipedia."
        summary = wikipedia.summary(title=title, sentences=5, auto_suggest=False,
            redirect=True)
    except Exception as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Wikipedia summary: {summary}"

summary = wikipedia_tool.invoke(input={"query": "SWIFT"})
print(summary)

In [ ]:
import typing
from langgraph import graph, prebuilt
import time

class MessagesState(typing.TypedDict):
    messages: typing.Annotated[list, graph.message.add_messages]
    llm_calls: int

workflow = graph.StateGraph(state_schema=MessagesState)

tools = [wikipedia_tool]
llm_with_tools = llm.bind_tools(tools=tools)

def llm_node(state: MessagesState):
    messages = llm_with_tools.invoke(input=state["messages"])
    return {"messages": [messages]}
workflow.add_node(node="llm", action=llm_node)

tools_node = prebuilt.ToolNode(tools=tools)
workflow.add_node(node="tools", action=tools_node)

workflow.add_edge(start_key=graph.START, end_key="llm")
workflow.add_edge(start_key="llm", end_key="tools")
workflow.add_edge(start_key="tools", end_key=graph.END)
agents = workflow.compile()
agents

In [ ]:
start_time = time.time()
response = agents.invoke(input={
    "messages": [{"role": "user", "content": "Tell me about SWIFT."}]
})
end_time = time.time() - start_time
print("Duration: %.2f, Response: %s"%(end_time, response))